## Poolish Baguette

- bulk fermentation: 2hrs, fold once
- divide & preshape: 10-30 min
- shape
- final fermentation: 1-1.5 hrs
- bake: ~25min


In [4]:
import pandas as pd
grams_per_ounce = 28.3495

In [5]:
num_loaves = 4
num_pounds = 0
num_ounces = 13.5
num_grams = 0

In [6]:
loaf_weight = (num_pounds * 16 + num_ounces) * grams_per_ounce + num_grams
total_weight = (num_loaves * loaf_weight)
print ("{:,.0f} loaves at {:,.0f} grams".
       format(num_loaves, loaf_weight, num_pounds, num_ounces))
print ("total weight: {:,.2f} grams".format(total_weight,total_weight * grams_per_ounce))

4 loaves at 383 grams
total weight: 1,530.87 grams


In [7]:
formula = pd.DataFrame.from_dict({
    "ap-flour": 100, 
    "water": 66, 
    "yeast": 0.7,
    "salt": 2,
    "malt": 0.5
}, orient='index', columns=["baker%"])
print ("overall formula")
formula_total = formula['baker%'].sum()
print("overall formula total = %s%%" % formula_total)
formula

overall formula
overall formula total = 169.2%


,baker%
ap-flour,100.0
water,66.0
yeast,0.7
salt,2.0
malt,0.5


In [8]:
pre_ferment = .33
poolish = pd.DataFrame.from_dict({
    "ap-flour": 100,
    "water": 100, 
    "yeast": 0.2
}, orient='index', columns=["baker%"])
print ("poolish")
poolish_total = poolish['baker%'].sum()
print("poolish total = %s%%" % poolish_total)
poolish

poolish
poolish total = 200.2%


,baker%
ap-flour,100.0
water,100.0
yeast,0.2


In [9]:
poolish_weight = total_weight * 100 / formula_total * pre_ferment * poolish_total/poolish.at['ap-flour','baker%']
print ("poolish weight = {:,.0f} grams".format(poolish_weight))
poolish['grams'] = poolish.apply(lambda row: '{:.0f}'.format(row['baker%']*poolish_weight/poolish_total), axis=1)
poolish['oz'] = poolish.apply(lambda row: '{:.1f}'.format(float(row['grams']) / grams_per_ounce), axis=1)
poolish = poolish[['baker%', 'grams', 'oz']]
poolish

poolish weight = 598 grams


,baker%,grams,oz
ap-flour,100.0,299,10.5
water,100.0,299,10.5
yeast,0.2,1,0.0


In [10]:
final_dough = formula
final_dough['grams'] = final_dough.apply(lambda row: '{:.0f}'.format(row['baker%']*total_weight / formula_total), axis=1)
final_dough.at['poolish','grams'] = '{:.0f}'.format(poolish_weight)
final_dough.at['poolish','baker%'] = '{:.0f}'.format(poolish_weight / float(final_dough.at['ap-flour','grams']) * 100)
for index, row in poolish.iterrows():
    final_dough.at[index,'grams'] = float(final_dough.at[index,'grams']) - float(row['grams'])
final_dough['oz'] = final_dough.apply(lambda row: '{:,.1f}'.format(float(row['grams'])/grams_per_ounce), axis=1)
final_dough = final_dough[['baker%', 'grams', 'oz']]
print("final dough")
final_dough

final dough


,baker%,grams,oz
ap-flour,100.0,606,21.4
water,66.0,298,10.5
yeast,0.7,5,0.2
salt,2.0,18,0.6
malt,0.5,5,0.2
poolish,66.0,598,21.1
